# GSM8k Experiments

In [3]:
import sys
import argparse

# Running with the same model pool as LLM-TOPLA.
parser = argparse.ArgumentParser()

parser.add_argument("--device", type=str, default="cuda")
parser.add_argument("--model_names", type=str, default="all")
parser.add_argument("--dataset_type", type=str, default="lang", choices=["lang", "vision"])
parser.add_argument("--task_name", type=str, default="gsm8k", choices=["gsm8k", "mmlu_hf", "bbh", "gpqa", "musr"])

# select agent arguments
parser.add_argument("--alpha", type=float, default=1.0, help="size penalty constant")
parser.add_argument("--select_agent_epoch", type=int, default=10)
parser.add_argument("--select_agent_gamma", type=float, default=0.99)
parser.add_argument("--select_agent_clip_epsilon", type=float, default=0.2)
parser.add_argument("--select_agent_lr", type=float, default=0.001)
parser.add_argument("--focal_div_weight", type=float, default=0.5)
parser.add_argument("--plurality_voting_weight", type=float, default=0.5)
parser.add_argument("--fleiss_kappa_weight", type=float, default=0)
parser.add_argument("--q_stats_weight", type=float, default=0)
parser.add_argument("--corr_coef_weight", type=float, default=0)
parser.add_argument("--binary_disagreement_weight", type=float, default=0)
parser.add_argument("--kappa_stats_weight", type=float, default=0)
parser.add_argument("--binary_entropy_weight", type=float, default=0)

# ensemble agent arguments
parser.add_argument("--ensemble_agent_epoch", type=int, default=10)
parser.add_argument("--ens_agent_gamma", type=float, default=0.5)
parser.add_argument("--ens_agent_clip_epsilon", type=float, default=0.2)
parser.add_argument("--ens_agent_lr", type=float, default=0.001)

# common argumentss for both agents
parser.add_argument("--window_size", type=int, default=500)
parser.add_argument("--max_tolerance", type=int, default=150)
parser.add_argument("--update_freq", type=int, default=100)


# Simulate command-line arguments
if 'ipykernel' in sys.modules:
    sys.argv = ['notebook.py']  # Replace with desired arguments
else:
    sys.argv = ['script.py']

# Parse the arguments
args = parser.parse_args()

In [7]:
from run import main

test_scores = []
for i in range(5):
    test_score = main(args)
    test_scores.append(test_score)

['Llama-2-13b-chat-hf', 'Llama-2-70b-chat-hf', 'Llama-2-7b-chat-hf', 'Mistral-7B-Instruct-v0.2', 'Mixtral-8x7B-Instruct-v0.1', 'gemma-2b', 'gemma-7b', 'phi-2']
Truncating the space size
Truncating the space size
Dataset-0: gsm8k


  4%|▎         | 100/2748 [00:12<05:37,  7.84it/s]


KeyboardInterrupt: 

In [ ]:
print(test_scores)